In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt

# Define the stock symbol and date range
stock_symbol = "AAPL"
start_date = "2020-01-01"
end_date = "2021-12-31"

# Fetch historical stock price data
stock_data = yf.download(stock_symbol, start=start_date, end=end_date)

# Calculate daily returns
stock_data['Daily_Return'] = stock_data['Adj Close'].pct_change().fillna(0)

# Calculate volatility (e.g., rolling standard deviation)
window = 21  # Adjust the window size as needed
stock_data['Volatility'] = stock_data['Daily_Return'].rolling(window=window).std()

# Drop rows with NaN values
stock_data.dropna(inplace=True)

# Select the 'Volatility' column as the target variable
target = stock_data['Volatility'].values.reshape(-1, 1)

# Normalize the target variable using Min-Max scaling
scaler = MinMaxScaler()
target_scaled = scaler.fit_transform(target)

# Define a function to create input sequences
def create_sequences(data, sequence_length):
    sequences = []
    target = []
    for i in range(len(data) - sequence_length):
        seq = data[i:i+sequence_length]
        label = data[i+sequence_length]
        sequences.append(seq)
        target.append(label)
    return np.array(sequences), np.array(target)

# Set the sequence length (e.g., 10 days)
sequence_length = 10

# Create input sequences and target values
X, y = create_sequences(target_scaled, sequence_length)

# Define the training and testing set sizes
train_size = int(0.8 * len(X))

# Split the data
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Create an LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(sequence_length, 1)))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32)

# Make predictions on the test set
predictions_scaled = model.predict(X_test)

# Inverse transform the predictions to get volatility values
predictions = scaler.inverse_transform(predictions_scaled)

# Plot the actual vs. predicted volatility
plt.figure(figsize=(12, 6))
plt.plot(predictions, label='Predicted Volatility', color='red')
plt.plot(y_test, label='Actual Volatility', color='blue')
plt.legend()
plt.title('Stock Volatility Prediction')
plt.xlabel('Time')
plt.ylabel('Volatility')
plt.show()